In [3]:
! pip install numpy==1.21

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 38.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.10.0 requires libclang>=13.0.0, which is not installed.
tensorflow 2.10.0 requires tensorflow-io-gcs-filesystem>=0.23.1, which is not installed.
tensorflow 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.1 which is incompatible.
tensorboard 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.1 which is incompatible.


In [5]:
!which numpy

which: no numpy in (/gpfs/commons/home/tlin/miniconda3/envs/tensorflow/bin:/opt/slurm/slurm-20.11.0/bin:/opt/slurm/pe2/slurm/centos7/bin:/opt/slurm/slurm-20.11.0/bin:/gpfs/commons/home/tlin/miniconda3/condabin:/gpfs/commons/home/tlin/.local/bin:/gpfs/commons/home/tlin/bin:/opt/slurm/slurm-20.11.0/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/usr/lpp/mmfs/bin:/opt/puppetlabs/bin:/usr/lpp/mmfs/bin:/usr/lpp/mmfs/bin)


In [1]:
import tensorflow as tf

2023-01-31 16:54:46.268817: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
# Make sure the GPU is enabled 
assert tf.config.list_physical_devices('GPU'), 'Start the colab kernel with GPU: Runtime -> Change runtime type -> GPU'

2023-01-31 16:55:23.609769: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-01-31 16:55:23.612845: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/slurm/slurm-20.11.0/lib64:/opt/slurm/pe2/slurm/centos7/lib64:/opt/slurm/slurm-20.11.0/lib64:/opt/slurm/slurm-20.11.0/lib64:
2023-01-31 16:55:23.612919: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-31 16:55:23.613018: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pe2cc3-043.c.nygenome.org): /proc/driver/nvidia/version does not exist


AssertionError: Start the colab kernel with GPU: Runtime -> Change runtime type -> GPU

In [3]:
tf.config.list_physical_devices

<function tensorflow.python.framework.config.list_physical_devices(device_type=None)>